<a href="https://colab.research.google.com/github/Sajag28/Prompt-Model-Refine/blob/main/LLM_Prompt_Instances_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install --upgrade pip
%pip install --disable pip version check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.1 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

In [5]:
huggingface_dataset_name="knkarthick/dialogsum"
datset=load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
example_indices=[40,200]
dash_line='-'.join('' for x in range(100))
for i,index in enumerate(example_indices):
  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print('INPUT DIALOGUE:')
  print(datset['test'][index]['dialogue'])
  print(dash_line)
  print('BASELINE HUMAN SUMMARY:')
  print(datset['test'][index]['summary'])
  print(dash_line)
  print()


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exam

In [9]:
model_name="google/flan-t5-base"
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [12]:
#Simple encoding of the sentence
sentence="What time is it, Sajag?"
sentence_encoded=tokenizer(sentence,return_tensors='pt')
sentence_decoded=tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens=True
)
print('ENCODED SENTENCE: ')
print(sentence_encoded["input_ids"][0])
print('DECODED SENTENCE: ')
print(sentence_decoded)

ENCODED SENTENCE: 
tensor([ 363,   97,   19,   34,    6, 1138, 1191,  122,   58,    1])
DECODED SENTENCE: 
What time is it, Sajag?


In [13]:
#Without any Shot Instance
for i,index in enumerate(example_indices):
  dialogue=datset['test'][index]['dialogue']
  summary=datset['test'][index]['summary']

  inputs=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50,

      )[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue} ')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION-WITHOUT PROMPT ENGINEERING\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there. 
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION-WITHOUT PROMPT ENGINEERING
Person1: It's ten to nine.

-----------------------------------

In [14]:
#Zero Shot Instance by training by using two examples
for i,index in enumerate(example_indices):
  dialogue=datset['test'][index]['dialogue']
  summary=datset['test'][index]['summary']

  prompt=f"""
  Summarize the following conversations.

  {dialogue}

  Summary:
  """
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION-ZERO SHOT:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Summarize the following conversations.

  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  Summary:
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION-ZERO SHOT:
The train is about

In [15]:
def make_prompt(example_indices_full,example_index_to_summarize):
  prompt=''
  for index in example_indices_full:
    dialogue=datset['test'][index]['dialogue']
    summary=datset['test'][index]['summary']

    prompt+=f"""
    Dialogue:
    {dialogue}

    What was going on?
    {summary}

    """
    dialogue=datset['test'][example_index_to_summarize]['dialogue']
    prompt+=f"""
    Dialogue:

    {dialogue}

    What was going on?
    """
    return prompt



In [16]:
example_indices_full=[40]
example_index_to_summarize=200
one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)

print(one_shot_prompt)


    Dialogue:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What was going on?
    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

    
    Dialogue:

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to b

In [17]:
summary=datset['test'][example_index_to_summarize]['summary']

inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
        )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION-ONE SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION-ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [19]:
#Few Shot Insrances by using few examples to train the prompt model
example_indices_full=[40,80,120]
example_index_to_summarize=200
few_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)

print(few_shot_prompt)



    Dialogue:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What was going on?
    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

    
    Dialogue:

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to b

In [20]:
summary=datset["test"][example_index_to_summarize]['summary']
inputs=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION-FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION-FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
